# Notes
* Shared Weights (Pos2Vec)
* Learning rate 0.01 * 0.99 after each epoch
* 1000 Epochs
* 1,000,000 W - 1,000,000 L. Total 1,000,000 pairs.

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Concatenate
from datetime import datetime
import numpy as np
import pandas as pd

import gc
from tensorflow.keras import backend as k


In [2]:
NUM_EPOCHS = 1000

In [3]:
def reset_random_seeds():
    '''
    Sets all necessary seed for reproduceability.
    '''
    os.environ['PYTHONHASHSEED']=str(1)
    tf.random.set_seed(1)
    np.random.seed(1)
    
reset_random_seeds()

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# Batch Creation

In [5]:
# Est 2mins 45secs
# win = np.load('../data/win_loss/win.npy')
# loss = np.load('../data/win_loss/loss.npy')
# win = win[:3000000]
# loss = loss[:3000000]
# np.save('../data/win_loss/win_3mil.npy', win)
# np.save('../data/win_loss/loss_3mil.npy', loss)

# # Est 2mins
# # Reduced size of wins and losses due to memory constraint
# win = np.load('../data/win_loss/win_3mil.npy')
# loss = np.load('../data/win_loss/loss_3mil.npy')

# Est 1min 20sec
# Reduced size of wins and losses to Deep Chess's due to memory constraint.
# win = win[:2216950]
# loss = loss[:1643870]
# np.save('../data/win_loss/win_deepchess.npy', win)
# np.save('../data/win_loss/loss_deepchess.npy', loss)
win = np.load('../data/win_loss/win_deepchess.npy')
loss = np.load('../data/win_loss/loss_deepchess.npy')


In [6]:
len(win), len(loss)

(2216950, 1643870)

In [7]:
model = Sequential([
    Dense(773,activation='relu'),
    Dense(600,activation='relu'),
    Dense(400,activation='relu'),
    Dense(200,activation='relu'),
    Dense(100,activation='relu')
])
model.load_weights('../models/fpos2vec_v1/fpos2vec_v1')

In [8]:
left_input = Input((None, 773))
right_input = Input((None, 773))
encoded_l = model(left_input)
encoded_r = model(right_input)

In [9]:
concatenated = Concatenate()([encoded_l, encoded_r])
deepchess = Dense(400, activation='relu')(concatenated)
deepchess = Dense(200, activation='relu')(deepchess)
deepchess = Dense(100, activation='relu')(deepchess)
deepchess_output = Dense(2, activation='softmax')(deepchess)

deepchess_model = Model(inputs=[left_input, right_input], outputs=deepchess_output)
deepchess_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=tf.keras.losses.binary_crossentropy, metrics=['acc']) #, run_eagerly=True)

In [10]:
def scheduler(epoch, lr):
    return lr*0.99
reduce_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [11]:
class ClearMemory(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        k.clear_session()

In [12]:
def get_train_data(train_size=1000000):
    # Get wins and losses
    w_selection = np.random.choice(win.shape[0], train_size, replace=False)
    w = win[w_selection]
    l_selection = np.random.choice(loss.shape[0], train_size, replace=False)
    l = loss[l_selection]

    # Get entire dataframe (1.2million rows)
    df = pd.DataFrame(data=[(w[i], l[i], np.array([1,0]).astype('uint8')) for i in range(train_size)],columns=['x_left', 'x_right', 'y'])

    # Find random rows where we swap win/loss to loss/win
    idx = np.random.rand(len(df)) < 0.5
    # Update swapped rows
    df.loc[idx, ['x_left', 'x_right']] = df.loc[idx, ['x_right', 'x_left']].to_numpy()
    # x_val_left, x_val_right, y_val
    return np.array(df.x_left.to_list())[:, np.newaxis, :], np.array(df.x_right.to_list())[:, np.newaxis, :], np.array([(np.array([0,1]).astype('uint8')) if i else (np.array([1,0]).astype('uint8')) for i in idx])[:, np.newaxis, :]


In [13]:
def get_val_data(test_size=200000):
    # Get wins and losses
    w_selection = np.random.choice(win.shape[0], test_size, replace=False)
    w = win[w_selection]
    l_selection = np.random.choice(loss.shape[0], test_size, replace=False)
    l = loss[l_selection]

    # Get entire dataframe (1.2million rows)
    df = pd.DataFrame(data=[(w[i], l[i], np.array([1,0]).astype('uint8')) for i in range(test_size)],columns=['x_left', 'x_right', 'y'])

    # Find random rows where we swap win/loss to loss/win
    idx = np.random.rand(len(df)) < 0.5
    # Update swapped rows
    df.loc[idx, ['x_left', 'x_right']] = df.loc[idx, ['x_right', 'x_left']].to_numpy()
    # x_val_left, x_val_right, y_val
    return np.array(df.x_left.to_list())[:, np.newaxis, :], np.array(df.x_right.to_list())[:, np.newaxis, :], np.array([(np.array([0,1]).astype('uint8')) if i else (np.array([1,0]).astype('uint8')) for i in idx])[:, np.newaxis, :]


    # w = win[np.random.choice(win.shape[0], test_size, replace=False)]
    # # l_selection = np.random.choice(loss.shape[0], train_size, replace=False)
    # l = loss[np.random.choice(loss.shape[0], test_size, replace=False)]
    # y = np.array([[1,0]]*len(w)).astype('uint8')
    # for i,_ in enumerate(np.random.rand(len(w)) < 0.5):
    #     w[i], l[i] = l[i], w[i]
    #     y[i] = [0,1]
    # return w[:, np.newaxis, :], l[:, np.newaxis, :], y[:, np.newaxis, :]

In [14]:
periodic_save = tf.keras.callbacks.ModelCheckpoint(filepath='../models/deepchess/deepchess', verbose=0,
                    save_best_only=True, monitor='val_acc', mode='max', save_weights_only=False)

In [15]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(
        self, 
        x_left,
        x_right,
        y,
        batch_size=50000, 
        num_classes=None,
    ):
        self.x_left = x_left
        self.x_right = x_right
        self.y = y
        self.batch_size = batch_size
        self.num_classes = num_classes
        
    def __len__(self):
        # Denotes the number of batches per epoch
        return len(self.y) // self.batch_size

    def __getitem__(self, index):
        return (self.x_left[index*self.batch_size:(index+1)*self.batch_size], self.x_right[index*self.batch_size:(index+1)*self.batch_size]), self.y[index*self.batch_size:(index+1)*self.batch_size]

In [16]:
history = []
for epoch in range(NUM_EPOCHS):
    print(f'Epoch {epoch}: {datetime.now().strftime("%d/%m/%Y %H:%M:%S")} | ', end='')
    x_train_left, x_train_right, y_train = get_train_data()
    x_val_left, x_val_right, y_val = get_val_data()
    
    train_datagen = DataGenerator(x_train_left, x_train_right, y_train)
    val_datagen = DataGenerator(x_val_left, x_val_right, y_val, batch_size=25000)
    
    deepchess_model.fit(x=train_datagen, callbacks=[reduce_lr, ClearMemory(), periodic_save], validation_data=val_datagen, verbose=0, max_queue_size=1)
    
    history.append(deepchess_model.history.history)
    print(f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")}: {deepchess_model.history.history}')

Epoch 0: 25/10/2021 04:17:27 | WARNING:tensorflow:From C:\Users\lowbe\Anaconda3\envs\fyp\lib\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../models/deepchess\deepchess\assets
25/10/2021 04:17:56: {'loss': [0.7619384527206421], 'acc': [0.5582860112190247], 'val_loss': [0.5744996666908264], 'val_acc': [0.6994749903678894], 'lr': [0.0099]}
Epoch 1: 25/10/2021 04:17:56 | INFO:tensorflow:Assets written to: ../models/deepchess\deepchess\assets
25/10/2021 04:18:22: {'loss': [0.5092024207115173], 'acc': [0.7399420142173767], 'val_loss': [0.4690490961074829], 'val_acc': [0.7646600008010864

In [17]:
deepchess_model.save_weights('../models/deepchess/deepchess_weights')

In [1]:
import tensorflow as tf
finalmodel = tf.keras.models.load_model('../models/deepchess/deepchess')

In [2]:
finalmodel.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 773)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 773)]  0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, None, 100)    1403402     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, None, 200)    0           sequential[0][0]      

In [21]:
import random
x = random.sample(list(zip(win, loss)), 5)
x=np.array(x)

In [36]:
l=[]
r=[]
for item in x:
    l.append(item[0])
    r.append(item[1])
l = np.array(l)
l = l[:, np.newaxis, :]

r = np.array(r)
r = r[:, np.newaxis, :]

l.shape, r.shape

((5, 1, 773), (5, 1, 773))

In [47]:
loaded_preds = finalmodel.predict((l,r))
original_preds = deepchess_model.predict((l,r))